In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectFromModel



In [3]:
df = pd.read_csv("C:\\Users\Ali\\DataScience_Projects\\Kaggle_Competition\\train_set.csv")
df.head()

,RecordId,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X70,X71,X72,X73,X74,X75,X76,X77,X78,Y
0,1,87.000000,34.118411,0,2,0,165.100000,1,829,2,...,0.040000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2,82.372284,31.573280,0,0,1,162.983897,1,724,0,...,0.033431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,3,50.000000,27.771653,0,0,1,165.100000,1,895,2,...,0.010000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,4,66.236109,26.515922,0,0,1,167.009549,1,637,0,...,0.039363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,5,81.303299,20.843691,0,0,1,158.165419,0,564,0,...,0.069242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [4]:
recordId = df['RecordId']
test_data = df.drop(columns = 'RecordId')

In [5]:
# Sample data
data = df

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the data
df = scaler.fit_transform(data)

In [7]:
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df)
imputed_data = imr.transform(df)
df= pd.DataFrame(imputed_data)

In [8]:
# Separating the target variable
X = df.values[:, 1:78] # 1 - 78 is 77 features... (78 is exclusive)
Y = df.values[:, 78]

# Splitting the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.1, random_state=100)


In [9]:
model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=100)

In [10]:
selector = SelectFromModel(estimator=model, prefit=False)
selector.fit(X_train,y_train)


SelectFromModel(estimator=GradientBoostingClassifier(random_state=100))

In [11]:
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

In [12]:
model.fit(X_train_selected, y_train)

GradientBoostingClassifier(random_state=100)

In [13]:
y_pred = model.predict(X_test_selected)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

accuracy


0.9971559744850282

In [14]:
test_data = pd.read_csv("C:\\Users\Ali\\DataScience_Projects\\Kaggle_Competition\\test_set.csv")

In [15]:
recordId = test_data['RecordId']
test_data = test_data.drop(columns = 'RecordId')

In [16]:
# Sample data
data = test_data

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the data
test_data = scaler.fit_transform(data)

In [17]:
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(test_data)
imputed_data = imr.transform(test_data)
test_data = pd.DataFrame(imputed_data)

In [18]:
test_data = selector.transform(test_data)


In [19]:
prediction = model.predict_proba(test_data)
prediction= prediction[:,1]

In [20]:
finaldf = pd.concat([recordId, pd.DataFrame(prediction, columns=['Y'])], axis=1)
finaldf
finaldf.to_csv("C:\\Users\Ali\\DataScience_Projects\\Kaggle_Competition\\test_results.csv", index= False)